# 搭建并使用向量数据库

## 前序数据预处理

In [184]:
#!/usr/bin/env python
# coding: utf-8

# 批量处理文件夹中所有文件
import os

# 获取folder_path下所有文件路径，储存在file_paths里
file_paths = []
folder_path = '../data_base/knowledge_path/VMAX-S'
for root, dirs, files in os.walk(folder_path):
    for file in files:
        file_path = os.path.join(root, file)
        file_paths.append(file_path)
print(file_paths)

from langchain.document_loaders.pdf import PyMuPDFLoader
# from langchain.document_loaders.markdown import UnstructuredMarkdownLoader

# 遍历文件路径并把实例化的loader存放在loaders里
loaders = []

for file_path in file_paths:
    file_type = file_path.split('.')[-1]
    if file_type == 'pdf':
        loaders.append(PyMuPDFLoader(file_path))
    else:
        print(f"Unsupported file type: {file_type} for file {file_path}")

# 下载文件并存储到text
# 加载所有文档内容到 texts
texts = []
for loader in loaders:
    texts.extend(loader.load())  # 关键步骤：初始化 texts

    
# 作数据清洗
# 修改后的数据清洗部分（替换原始代码中对应段落）
import re

# 预编译正则表达式（提升效率）
linebreak_pattern = re.compile(
    r'(?<![\\u4e00-\\u9fff])\n(?![\\u4e00-\\u9fff])',  # 负向断言匹配非中文环境换行
    flags=re.DOTALL
)
space_pattern = re.compile(r'[ 　]+')  # 匹配半角/全角空格
special_chars = ['•', '▪', '▫', '▶', '®', '©']  # 可扩展的干扰符号列表

# 替换原始代码中的清洗循环
for text in texts:
    # 1. 清理非中文环境换行
    text.page_content = re.sub(
        linebreak_pattern,
        lambda m: m.group().replace('\n', ''),
        text.page_content
    )

    # 2. 批量清理特殊符号
    for char in special_chars:
        text.page_content = text.page_content.replace(char, '')

    # 3. 安全删除空格（保留URL等特殊场景）
    text.page_content = space_pattern.sub('', text.page_content)



from dotenv import load_dotenv, find_dotenv 
# pip install python-dotenv

_ = load_dotenv(find_dotenv())

from langchain_community.embeddings import ZhipuAIEmbeddings

zhipu_embed = ZhipuAIEmbeddings(
    model="embedding-2",
    api_key="5713143e8fdc4b4a8b284cf97092e70f.qEK71mGIlavzO1Io",
)


''' 
* RecursiveCharacterTextSplitter 递归字符文本分割
RecursiveCharacterTextSplitter 将按不同的字符递归地分割(按照这个优先级["\n\n", "\n", " ", ""])，
    这样就能尽量把所有和语义相关的内容尽可能长时间地保留在同一位置
RecursiveCharacterTextSplitter需要关注的是4个参数：

* separators - 分隔符字符串数组
* chunk_size - 每个文档的字符数量限制
* chunk_overlap - 两份文档重叠区域的长度
* length_function - 长度计算函数
'''
#导入文本分割器
from langchain_text_splitters import RecursiveCharacterTextSplitter



# 知识库中单段文本长度
CHUNK_SIZE = 512

# 知识库中相邻文本重合长度
OVERLAP_SIZE = 50


# 使用递归字符文本分割器
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=OVERLAP_SIZE
)


split_docs = text_splitter.split_documents(texts)
print(f"切分后的文件数量：{len(split_docs)}")



print(f"切分后的字符数（可以用来大致评估 token 数）：{sum([len(doc.page_content) for doc in split_docs])}")



split_docs[90].page_content



['../data_base/knowledge_path/VMAX-S\\SJ-20130403110434-005-ZXCLOUD E9000（V1.0）刀片服务器 快速安装指南.pdf', '../data_base/knowledge_path/VMAX-S\\SJ-20140307103120-034-ZXCLOUD E9000（V1.0）刀片服务器 硬件安装指导.pdf', '../data_base/knowledge_path/VMAX-S\\SJ-20140326173355-003-ZXCLOUD I8350 G2（V1.0）机架服务器 硬件安装指导.pdf', '../data_base/knowledge_path/VMAX-S\\SJ-20151221160747-002-ZXVMAX（V6.15）多维价值分析系统 软件安装指导.pdf', '../data_base/knowledge_path/VMAX-S\\SJ-20151221160747-011-ZXVMAX（V6.15）多维价值分析系统 软件安装指导（CDMA分册）.pdf', '../data_base/knowledge_path/VMAX-S\\SJ-20151225170610-002-ZXVMAX（V6.15）多维价值分析系统 主题表安装指南（LTE分册）.pdf', '../data_base/knowledge_path/VMAX-S\\ZXVMAX-S（V6.19.20.10.03）硬件安装指导.pdf', '../data_base/knowledge_path/VMAX-S\\ZXVMAX-S（V6.23）安全加固指导.pdf']
切分后的文件数量：695
切分后的字符数（可以用来大致评估 token 数）：236550


'4安装设备图4-6固定导轨支架\n6.将中导轨拉出到锁定位置，并确保中导轨被锁定（A处锁定），如图4-7所示。说说说明明明：：：在装入服务器之前请确保滚珠座B在滑轨前段位置。图4-7拉出中导轨\n7.将安装好内导轨的服务器沿着中导轨前端卡入，卡好后向前推入，如图4-8所示。说说说明明明：：：在将导轨卡入到中轨并推入的过程中，须注意将内导轨卡入到中导轨的滚珠中。\nSJ-20140326173355-003|2014-04-01（R1.0）\n4-5'

## 前序embedding模型

ZhipuAIEmbeddings模型

In [185]:
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

from langchain_community.embeddings import ZhipuAIEmbeddings

zhipu_embed = ZhipuAIEmbeddings(
    model="embedding-2",
    api_key = os.environ['ZHIPUAI_API_KEY'],
)

批量处理文件夹中所有文件

## 构建Chroma向量库

涉及到的API

In [186]:
# langchain_chroma.vectordbs.Chroma
# class langchain_chroma.vectordbs.Chroma(
#     collection_name: str = 'langchain', 
# embedding_function: Embeddings | None = None, 
# persist_directory: str | None = None, 
# client_settings: Settings | None = None, 
# collection_metadata: Dict | None = None, 
# client: ClientAPI | None = None, 
# relevance_score_fn: Callable[[float], float] | None = None, 
# create_collection_if_not_exists: bool | None = True)


# classmethod from_documents(
#     documents: List[Document], 
# embedding: Embeddings | None = None, 
# ids: List[str] | None = None, 
# collection_name: str = 'langchain', 
# persist_directory: str | None = None, 
# client_settings: Settings | None = None, 
# client: ClientAPI | None = None, 
# collection_metadata: Dict | None = None, 
# **kwargs: Any)

直接调用Chroma.from_documents (推荐)

In [193]:
# 汇总

import os
from langchain_community.vectorstores import Chroma
# from langchain_chroma.vectorstores import Chroma
from langchain_community.embeddings import ZhipuAIEmbeddings

# 定义持久化目录
persist_directory = '../chroma-vmax-7'

# 创建嵌入模型
embedding = ZhipuAIEmbeddings(model="embedding-2", api_key = os.environ['ZHIPUAI_API_KEY'])

try:
    # 初始化 Chroma 向量数据库
    vectordb = Chroma.from_documents(
        documents=split_docs[:20],  # 为了速度，只选择前 20 个切分的 doc 进行生成
        # documents=split_docs,  # 为了速度，只选择前 20 个切分的 doc 进行生成
        embedding=embedding,
        # collection_name="test1", # 如果不指定默认为langchain
        persist_directory=persist_directory, # 允许我们将persist_directory目录保存到磁盘上
    )
    
    # 持久化向量数据库
    vectordb.persist()
    print("向量数据库已成功持久化到磁盘。")
except Exception as e:
    print(f"持久化过程中发生错误: {e}")

向量数据库已成功持久化到磁盘。


以上代码会报错： 持久化过程中发生错误: Error code: 400, with error text {"error":{"code":"1214","message":"input数组最大不得超过64条"}}

建议分批次导入

In [188]:
import os
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import ZhipuAIEmbeddings

# 定义持久化目录
persist_directory = '../chroma-vmax'

# 创建嵌入模型
embedding = ZhipuAIEmbeddings(model="embedding-2", api_key=os.environ['ZHIPUAI_API_KEY'])

# 定义每批处理的文档数量
batch_size = 30

try:
    # 计算总批次数
    total_batches = (len(split_docs) + batch_size - 1) // batch_size
    
    # 初始化向量数据库（如果是第一次创建）
    vectordb = None
    
    for batch_num in range(total_batches):
        # 计算当前批次的起始和结束索引
        start_idx = batch_num * batch_size
        end_idx = min((batch_num + 1) * batch_size, len(split_docs))
        
        # 获取当前批次的文档
        batch_docs = split_docs[start_idx:end_idx]
        
        print(f"正在处理第 {batch_num + 1}/{total_batches} 批文档 (文档 {start_idx}-{end_idx-1})")
        
        if batch_num == 0:
            # 第一次创建向量数据库
            vectordb = Chroma.from_documents(
                collection_name='vmax-s',
                documents=batch_docs,
                embedding=embedding,
                persist_directory=persist_directory
            )
        else:
            # 后续批次添加到现有集合
            vectordb.add_documents(batch_docs)
        
        # 每批处理后持久化
        vectordb.persist()
        print(f"第 {batch_num + 1} 批文档已成功导入并持久化")
    
    print("所有文档已成功导入并持久化到向量数据库。")
    
except Exception as e:
    print(f"处理过程中发生错误: {e}")

正在处理第 1/24 批文档 (文档 0-29)
第 1 批文档已成功导入并持久化
正在处理第 2/24 批文档 (文档 30-59)
第 2 批文档已成功导入并持久化
正在处理第 3/24 批文档 (文档 60-89)
第 3 批文档已成功导入并持久化
正在处理第 4/24 批文档 (文档 90-119)
第 4 批文档已成功导入并持久化
正在处理第 5/24 批文档 (文档 120-149)
第 5 批文档已成功导入并持久化
正在处理第 6/24 批文档 (文档 150-179)
第 6 批文档已成功导入并持久化
正在处理第 7/24 批文档 (文档 180-209)
第 7 批文档已成功导入并持久化
正在处理第 8/24 批文档 (文档 210-239)
第 8 批文档已成功导入并持久化
正在处理第 9/24 批文档 (文档 240-269)
第 9 批文档已成功导入并持久化
正在处理第 10/24 批文档 (文档 270-299)
第 10 批文档已成功导入并持久化
正在处理第 11/24 批文档 (文档 300-329)
第 11 批文档已成功导入并持久化
正在处理第 12/24 批文档 (文档 330-359)
第 12 批文档已成功导入并持久化
正在处理第 13/24 批文档 (文档 360-389)
第 13 批文档已成功导入并持久化
正在处理第 14/24 批文档 (文档 390-419)
第 14 批文档已成功导入并持久化
正在处理第 15/24 批文档 (文档 420-449)
第 15 批文档已成功导入并持久化
正在处理第 16/24 批文档 (文档 450-479)
第 16 批文档已成功导入并持久化
正在处理第 17/24 批文档 (文档 480-509)
第 17 批文档已成功导入并持久化
正在处理第 18/24 批文档 (文档 510-539)
第 18 批文档已成功导入并持久化
正在处理第 19/24 批文档 (文档 540-569)
第 19 批文档已成功导入并持久化
正在处理第 20/24 批文档 (文档 570-599)
第 20 批文档已成功导入并持久化
正在处理第 21/24 批文档 (文档 600-629)
第 21 批文档已成功导入并持久化
正在处理第 22/24 批文档 (文档 630-659)
第 22 批文档已成

charma服务器版本

In [ ]:
import chromadb
import chromadb.utils.embedding_functions as embedding_functions

chroma_client = chromadb.HttpClient(host='localhost', port=8000)

openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key = os.environ['ZHIPUAI_API_KEY'],
                model_name="embedding-2"
            )
collection = chroma_client.get_or_create_collection(
    name="test4",
    embedding_function=openai_ef,  # 使用openai
    metadata={
        "hnsw:space": "cosine", 
        "hnsw:construction_ef": 150,
        "hnsw:M": 24,
        "hnsw:num_threads": 8
    }
)
ids_list = [f"doc_{i+1}" for i in range(len(split_docs[:20]))]
print(ids_list)

try:
    # 添加文档到集合
    collection.add(documents=split_docs[:20], ids =ids_list )
    # 持久化向量数据库
    vectordb.persist()
    print("向量数据库已成功持久化到磁盘。")
except Exception as e:
    print(f"持久化过程中发生错误: {e}")

通过chroma数据库add

按官网文档进行操作

In [13]:
content_list = ['本发布日期南书', 
                '前言周志华老师的《机器学习》西瓜书是机器学习领域的经典入门教材之一，周老师为了使尽可能多的读者通过西瓜书对机器学习有所了解所以在书中对部分公式的推导细节没有详述，但是这对那些想深究公式推导细节的读者来说可能不太友好，本书旨在对西瓜书里比较难理解的公式加以解析，以及对部分公式补充具体的推导细节。读到这里，大家可能会疑问为啥前面这段话加了引号，因为这只是我们最初的遐想，后来我们了解到，周老师之所以省去这些推导细节的真实原因是，他本尊认为理工科数学基础扎实点的大二下学生应该对西瓜书中的推导细节无困难吧，要点在书里都有了，略去的细节应能脑补或做练习。所以本南瓜书只能算是我等数学渣渣在自学的时候记下来的笔记，希望能够帮助大家都成为一名合格的理工科数学基础扎实点的大二下学生。使用说明南瓜书的所有内容都是以西瓜书的内容为前置知识进行表述的，所以南瓜书的最佳使用方法是以西瓜书为主线，遇到自己推导不出来或者看不懂的公式时再来查阅南瓜书；对于初学机器学习的小白，西瓜书第章和第章的公式强烈不建议深究，简单过一下即可，等你学得', 
                '有点飘的时候再回来啃都来得及；每个公式的解析和推导我们都力争以本科数学基础的视角进行讲解，所以超纲的数学知识我们通常都会以附录和参考文献的形式给出，感兴趣的同学可以继续沿着我们给的资料进行深入学习；若南瓜书里没有你想要查阅的公式，或者你发现南瓜书哪个地方有错误，请毫不犹豫地去我们的地址：进行反馈，在对应版块提交你希望补充的公式编号或者勘误信息，我们通常会在小时以内给您回复，超过小时未回复的话可以微信联系我们微信号：；配套视频教程：在线阅读地址：仅供第版最新版获取地址：编委会', 
                '编委会主编：、、编委：、、、、封面设计：构思、创作林王茂盛致谢特别感谢、、、、、、、、、、、在最早期的时候对南瓜书所做的贡献。扫描下方二维码，然后回复关键词南瓜书，即可加入南瓜书读者交流群版权声明本作品采用知识共享署名非商业性使用相同方式共享国际许可协议进行许可。'
               ]
print(len(content_list))

4


In [ ]:
collection.add(
    documents=["doc1", "doc2", "doc3", ...],
    embeddings=[[1.1, 2.3, 3.2], [4.5, 6.9, 4.4], [1.1, 2.3, 3.2], ...],
    metadatas=[{"chapter": "3", "verse": "16"}, {"chapter": "3", "verse": "5"}, {"chapter": "29", "verse": "11"}, ...],
    ids=["id1", "id2", "id3", ...]
)


In [20]:
from chromadb.utils import embedding_functions
import os
# from langchain_community.vectorstores import Chroma
import chromadb
from langchain_community.embeddings import ZhipuAIEmbeddings
from chromadb import PersistentClient
# 定义持久化目录
persist_directory = '../chroma-vmax'
# 初始化客户端（持久化模式）
client = PersistentClient(path=persist_directory)

import chromadb.utils.embedding_functions as embedding_functions
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key = os.environ['ZHIPUAI_API_KEY'],
                model_name="embedding-2"
            )
# collection = client.get_or_create_collection(
#     name="test",
#     embedding_function=openai_ef,  # 使用openai
#     metadata={
#         "hnsw:space": "cosine", 
#         "hnsw:construction_ef": 150,
#         "hnsw:M": 24,
#         "hnsw:num_threads": 8
#     }
# )

# HNSW索引构建参数优化
collection = client.create_collection(
    name="high_perf",
    metadata={
        "hnsw:construction_ef": 200,  # 构建阶段搜索范围（默认100）
        "hnsw:search_ef": 320,        # 查询阶段搜索范围 
        "hnsw:M": 48                  # 图节点最大连接数
    }
)

content_list = ['本发布日期南书', 
                '前言周志华老师的《机器学习》西瓜书是机器学习领域的经典入门教材之一，周老师为了使尽可能多的读者通过西瓜书对机器学习有所了解所以在书中对部分公式的推导细节没有详述，但是这对那些想深究公式推导细节的读者来说可能不太友好，本书旨在对西瓜书里比较难理解的公式加以解析，以及对部分公式补充具体的推导细节。读到这里，大家可能会疑问为啥前面这段话加了引号，因为这只是我们最初的遐想，后来我们了解到，周老师之所以省去这些推导细节的真实原因是，他本尊认为理工科数学基础扎实点的大二下学生应该对西瓜书中的推导细节无困难吧，要点在书里都有了，略去的细节应能脑补或做练习。所以本南瓜书只能算是我等数学渣渣在自学的时候记下来的笔记，希望能够帮助大家都成为一名合格的理工科数学基础扎实点的大二下学生。使用说明南瓜书的所有内容都是以西瓜书的内容为前置知识进行表述的，所以南瓜书的最佳使用方法是以西瓜书为主线，遇到自己推导不出来或者看不懂的公式时再来查阅南瓜书；对于初学机器学习的小白，西瓜书第章和第章的公式强烈不建议深究，简单过一下即可，等你学得', 
                '有点飘的时候再回来啃都来得及；每个公式的解析和推导我们都力争以本科数学基础的视角进行讲解，所以超纲的数学知识我们通常都会以附录和参考文献的形式给出，感兴趣的同学可以继续沿着我们给的资料进行深入学习；若南瓜书里没有你想要查阅的公式，或者你发现南瓜书哪个地方有错误，请毫不犹豫地去我们的地址：进行反馈，在对应版块提交你希望补充的公式编号或者勘误信息，我们通常会在小时以内给您回复，超过小时未回复的话可以微信联系我们微信号：；配套视频教程：在线阅读地址：仅供第版最新版获取地址：编委会', 
                '编委会主编：、、编委：、、、、封面设计：构思、创作林王茂盛致谢特别感谢、、、、、、、、、、、在最早期的时候对南瓜书所做的贡献。扫描下方二维码，然后回复关键词南瓜书，即可加入南瓜书读者交流群版权声明本作品采用知识共享署名非商业性使用相同方式共享国际许可协议进行许可。'
               ]
# print(len(content_list))

# 如果每个文档块的元数据需要个性化（例如记录分块编号），可以生成与文档数量匹配的元数据列表：
# metadatas = [{"source": "权威文档.pdf", "chunk_id": i} for i in range(len(split_docs[:20]))]

# 如果所有文档块的元数据相同（例如来源一致），可通过列表推导式生成等长元数据列表：
metadatas = [{"source": "权威文档.pdf"} for _ in range(len(split_docs[:20]))]

ids_list = [f"doc_{i+1}" for i in range(len(content_list))]
# print(ids_list)


# print(metadatas)

try:
    # 添加文档到集合（修正后的参数）
    collection.add(
        documents=content_list,  # 传入纯文本列表
        metadatas=[{"source": "权威文档.pdf"},{"source": "权威文档.pdf"},{"source": "权威文档.pdf"},{"source": "权威文档.pdf"} ],
        ids=ids_list
    )
    print("数据写入成功，已自动持久化到磁盘。")
except Exception as e:
    print(f"持久化过程中发生错误: {e}")
except chromadb.errors.ChromaError as ce:
    print(f"数据库错误: {ce}")
except ValueError as ve:
    print(f"数据格式错误: {ve}")

C:\Users\will\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|███████| 79.3M/79.3M [05:29<00:00, 253kiB/s]


数据写入成功，已自动持久化到磁盘。


正式方法

In [123]:
from langchain_community.embeddings import OllamaEmbeddings
ollama_emb = OllamaEmbeddings(
    base_url='http://localhost:11434',
    model="nomic-embed-text:latest"
)
r1 = ollama_emb.embed_documents(
    [
        "Alpha is the first letter of Greek alphabet",
        "Beta is the second letter of Greek alphabet",
    ]
)
r2 = ollama_emb.embed_query(
    "What is the second letter of Greek alphabet"
)

# 打印结果
print("Document embeddings:")
for i, embedding in enumerate(r1):
    print(f"Document {i+1} embedding: {embedding[:10]}")

print("\nQuery embedding:")
print(f"Query embedding: {r2[:10]}")

Document embeddings:
Document 1 embedding: [-0.1225031167268753, 0.6253135800361633, -3.54880428314209, -2.0378825664520264, 1.4127538204193115, -0.17774423956871033, -0.39122408628463745, 0.49636057019233704, -0.22381550073623657, -0.19290287792682648]
Document 2 embedding: [0.377452552318573, 0.9425770044326782, -3.3817975521087646, -1.5095171928405762, 1.954257845878601, -0.23311898112297058, -0.04492451250553131, -0.3590812087059021, -0.6258770823478699, -0.5528176426887512]

Query embedding:
Query embedding: [0.5814239978790283, 0.08968696743249893, -3.2821342945098877, -1.6575878858566284, 1.7467120885849, -0.9489914774894714, -0.8352117538452148, -0.02159128151834011, -0.279095858335495, -0.3286992311477661]


In [121]:
from chromadb.utils.embedding_functions.ollama_embedding_function import OllamaEmbeddingFunction
import requests

ollama_ef = OllamaEmbeddingFunction(
    url="http://localhost:11434",
    model_name="qwen2.5:0.5b "
)

try:
    response = ollama_ef._session.post(
        ollama_ef._api_url,
        json={"model": "nnomic-embed-text:latest", "prompt": "Test"}
    )
    print("原始响应内容:", response.text)  # 打印原始响应
    embeddings = ollama_ef(["Test"])
except Exception as e:
    print(f"错误详情: {str(e)}")

原始响应内容: 404 page not found
错误详情: Extra data: line 1 column 5 (char 4)


In [116]:
from chromadb.utils.embedding_functions.ollama_embedding_function import OllamaEmbeddingFunction
ollama_ef = OllamaEmbeddingFunction(url="http://localhost:11434",model_name="nomic-embed-text:latest")
embeddings = ollama_ef("This is my first text to embed")

JSONDecodeError: Extra data: line 1 column 5 (char 4)

In [124]:
from chromadb.utils import embedding_functions
import os
# from langchain_community.vectorstores import Chroma
import chromadb
from langchain_community.embeddings import ZhipuAIEmbeddings
from chromadb import PersistentClient
# 定义持久化目录
persist_directory = '../chroma-vmax-3'
# 初始化客户端（持久化模式）
client = PersistentClient(path=persist_directory)

from chromadb.utils.embedding_functions.ollama_embedding_function import OllamaEmbeddingFunction
ollama_ef = OllamaEmbeddingFunction(url="http://localhost:11434",model_name="nomic-embed-text:latest")

collection = client.get_or_create_collection(
    name="vmax-s",
    embedding_function=ollama_ef,  # 可选，但建议保持一致性
    metadata={
        "hnsw:construction_ef": 200,  # 构建阶段搜索范围（默认100）
        "hnsw:search_ef": 320,        # 查询阶段搜索范围 
        "hnsw:M": 48                  # 图节点最大连接数
    }
)

# content_list = [doc.page_content for doc in split_docs[:20]]
content_list = [doc.page_content for doc in split_docs]

# 如果每个文档块的元数据需要个性化（例如记录分块编号），可以生成与文档数量匹配的元数据列表：
# metadatas = [{"source": "权威文档.pdf", "chunk_id": i} for i in range(len(split_docs[:20]))]

# 如果所有文档块的元数据相同（例如来源一致），可通过列表推导式生成等长元数据列表：
# metadatas = [{"source": "VMAX安装系列.pdf"} for _ in range(len(split_docs[:20]))]
metadatas = [{"source": "VMAX安装系列.pdf"} for _ in range(len(split_docs))]

# ids_list = [f"doc_{i+1}" for i in range(len(split_docs[:20]))]
ids_list = [f"doc_{i+1}" for i in range(len(split_docs))]
# print(ids_list)


# print(metadatas)

try:
    # 添加文档到集合（修正后的参数）
    collection.add(
        documents=content_list,  # 传入纯文本列表
        metadatas=metadatas, 
        ids=ids_list
    )
    print("数据写入成功，已自动持久化到磁盘。")
except Exception as e:
    print(f"持久化过程中发生错误: {e}")
except chromadb.errors.ChromaError as ce:
    print(f"数据库错误: {ce}")
except ValueError as ve:
    print(f"数据格式错误: {ve}")

持久化过程中发生错误: JSONDecodeError.__init__() missing 2 required positional arguments: 'doc' and 'pos'


通过chroma数据库add-改良版 
每次三个doc，分批次导入

In [90]:
# 汇总

import os
# from langchain.vectordbs.chroma import Chroma
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import ZhipuAIEmbeddings
from chromadb import PersistentClient
from chromadb import Documents, EmbeddingFunction, Embeddings
# 定义持久化目录
persist_directory = '../chroma-vmax-batch'
# 初始化客户端（持久化模式）
client = PersistentClient(path=persist_directory)

# 创建嵌入模型
# import chromadb.utils.embedding_functions as embedding_functions
# openai_ef = embedding_functions.OpenAIEmbeddingFunction(
#                 api_key = os.environ['ZHIPUAI_API_KEY'],
#                 model_name="embedding-2"
#             )

# import chromadb.utils.embedding_functions as embedding_functions
# cohere_ef  = embedding_functions.CohereEmbeddingFunction(api_key="Tahx1eySFbKvu9sTyTXrRLf59la3ZUG9vy02stRZ",  model_name="embed-multilingual-v3.0")

zhipu_ef = ZhipuAIAdapter(model_name="embedding-2", api_key="5713143e8fdc4b4a8b284cf97092e70f.qEK71mGIlavzO1Io")

# 创建集合并配置HNSW参数
# collection = client.create_collection( 
# collection = client.get_or_create_collection(
#     name="test4",
#     embedding_function=openai_ef,
#     metadata={
#         "hnsw:space": "cosine", 
#         "hnsw:construction_ef": 150,
#         "hnsw:M": 24,
#         "hnsw:num_threads": 8
#     }
# )

collection = client.get_or_create_collection(
    name="vmax-s",
    embedding_function=cohere_ef,  # 可选，但建议保持一致性
    metadata={
        "hnsw:construction_ef": 200,  # 构建阶段搜索范围（默认100）
        "hnsw:search_ef": 320,        # 查询阶段搜索范围 
        "hnsw:M": 48                  # 图节点最大连接数
    }
)
# content_list = [doc.page_content for doc in split_docs[:20]]
content_list = [doc.page_content for doc in split_docs]

# 如果每个文档块的元数据需要个性化（例如记录分块编号），可以生成与文档数量匹配的元数据列表：
# metadatas = [{"source": "权威文档.pdf", "chunk_id": i} for i in range(len(split_docs[:20]))]

# 如果所有文档块的元数据相同（例如来源一致），可通过列表推导式生成等长元数据列表：
# metadatas = [{"source": "VMAX安装系列.pdf"} for _ in range(len(split_docs[:20]))]
metadatas = [{"source": "VMAX安装系列.pdf"} for _ in range(len(split_docs))]

# ids_list = [f"doc_{i+1}" for i in range(len(split_docs[:20]))]
ids_list = [f"doc_{i+1}" for i in range(len(split_docs))]
# print(ids_list)

try:
    # 添加文档到集合
    # collection.add(documents=split_docs)

    from itertools import batched
    docs_batches = batched(content_list, 3)  # 分3个批次
    ids_batches = batched(ids_list, 3)

    for docs, ids in zip(docs_batches, ids_batches):
        collection.add(
            documents=content_list,  # 传入纯文本列表
            metadatas=metadatas, 
            ids=ids_list
        )
    
    # 持久化向量数据库
    vectordb.persist()
    print("向量数据库已成功持久化到磁盘。")
except Exception as e:
    print(f"持久化过程中发生错误: {e}")

持久化过程中发生错误: status_code: 429, body: status_code: 429, body: {'message': 'trial token rate limit exceeded, limit is 100000 tokens per minute'} in add.


## 连接chroma数据库

In [36]:
# 数据库路径 ./chroma2

In [8]:
import chromadb
chroma_client  = chromadb.PersistentClient(path="../chroma-vmax")

In [9]:
chroma_client.list_collections()

['vmax-s']

In [10]:
# 获取一个存在的Collection对象
collection = chroma_client.get_collection("vmax-s")

In [11]:
collection.count()  #  returns the number of items in the collection.

695

In [5]:
# collection.peek()  # returns a list of the first 10 items in the collection.

In [ ]:
# 获取所有数据（默认不返回嵌入向量）
all_data = collection.get()
print(all_data)


## 向量检索
### 3.1 相似度检索
Chroma的相似度搜索使用的是余弦距离，即：
$$
similarity = cos(A, B) = \frac{A \cdot B}{\parallel A \parallel \parallel B \parallel} = \frac{\sum_1^n a_i b_i}{\sqrt{\sum_1^n a_i^2}\sqrt{\sum_1^n b_i^2}}
$$
其中$a_i$、$b_i$分别是向量$A$、$B$的分量。

当你需要数据库返回严谨的按余弦相似度排序的结果时可以使用`similarity_search`函数。

使用Chroma.from_documents时，用这个方法： 

In [6]:
question="vmax"

In [12]:
sim_docs = vectordb.similarity_search(question,k=3)
print(f"检索到的内容数：{len(sim_docs)}")

AttributeError: 'Client' object has no attribute 'similarity_search'

In [168]:
for i, sim_doc in enumerate(sim_docs):
    print(f"检索到的第{i}个内容: \n{sim_doc.page_content[:200]}", end="\n--------------\n")

检索到的第0个内容: 
3.7加固后检查
3.7.1账⼾系统共安全加固检查安全加固指导
19-25
--------------
检索到的第1个内容: 
数据库数据安全加固体系数据库合规与漏洞参⻅第3.5章节
3.安全加固操作
3.1加固流程⾸次开局/新建⽹络、升级、周期性维护等的加固流程如下图所⽰。安全加固分为三个阶段：前期加固、现场加固、后期加固。
1)前期加固：主要是出⼚前的预加固，⽐如操作系统等，在初始版本的基础上保证将补丁安装⾄最新，合规配置符合基本要求等。
2)现场加固：现场安装配置的加固，由于现场客⼾有⾃⼰的⼯具扫描，会发现⼀些未符合
--------------
检索到的第2个内容: 
11..适适⽤⽤范范围围产品安全加固的⽬的是“通过配置、设置、协议限制来减⼩攻击⾯”（来源于《产品安全要求总则》），使得产品免于或减⼩所受⽹络安全攻击的影响。产品安全加固对象包括产品所使⽤的第三⽅软件（如操作系统等），以及部分⾃研模块。安全合加固置是通过对加固对象中安全相关的配置项进⾏合理的设置，减⼩对象的攻击⾯，例如关闭不必要的端⼝、不必要的系统服务等；安全漏洞补丁是通过对加固对象的漏洞缺陷通过
--------------


### 3.2 MMR检索
如果只考虑检索出内容的相关性会导致内容过于单一，可能丢失重要信息。

最大边际相关性 (`MMR, Maximum marginal relevance`) 可以帮助我们在保持相关性的同时，增加内容的丰富度。

核心思想是在已经选择了一个相关性高的文档之后，再选择一个与已选文档相关性较低但是信息丰富的文档。这样可以在保持相关性的同时，增加内容的多样性，避免过于单一的结果。

In [171]:
mmr_docs = vectordb.max_marginal_relevance_search(question,k=3)

In [172]:
for i, sim_doc in enumerate(mmr_docs):
    print(f"MMR 检索到的第{i}个内容: \n{sim_doc.page_content[:200]}", end="\n--------------\n")

MMR 检索到的第0个内容: 
3.7加固后检查
3.7.1账⼾系统共安全加固检查安全加固指导
19-25
--------------
MMR 检索到的第1个内容: 
数据库数据安全加固体系数据库合规与漏洞参⻅第3.5章节
3.安全加固操作
3.1加固流程⾸次开局/新建⽹络、升级、周期性维护等的加固流程如下图所⽰。安全加固分为三个阶段：前期加固、现场加固、后期加固。
1)前期加固：主要是出⼚前的预加固，⽐如操作系统等，在初始版本的基础上保证将补丁安装⾄最新，合规配置符合基本要求等。
2)现场加固：现场安装配置的加固，由于现场客⼾有⾃⼰的⼯具扫描，会发现⼀些未符合
--------------
MMR 检索到的第2个内容: 
6安装机柜图6-38安装抗震加固组件（2）
1.横架板
2.竖架板
3.机房走线架
4.内六角螺栓M12×25/M8×25（含弹簧垫圈、垫片和绝缘垫圈）
5.绝缘垫片
6.锁紧杆（含六角螺母M8、弹簧垫圈和垫片）
7.螺栓（含螺母M8、弹簧垫圈和垫片）
SJ-20190815094124-009|2020-07-30（R1.0）
6-33
--------------
